In [1]:
import numpy as np
import pandas as pd
import tabula

Read and OCR the 20th page

In [2]:
table = tabula.read_pdf("adsi-2015-full-report.pdf",pages=20)

Got stderr: Nov 27, 2020 12:19:56 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:19:56 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:19:56 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:19:59 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:19:59 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Nov 27, 2020 12:19:59 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



Gain understanding of extra text read

In [3]:
table2 = table[0].copy()
table[0].head()

,Unnamed: 0,Unnamed: 1,Number of Accidental Deaths & Sui cides in India During 1967 to 2015,Unnamed: 2
0,Sl.,NaN,Accidental Deaths Suicides,NaN
1,No.,Year,Male Female Transgender Total Male Female Tran...,Total
2,(1),(2),(3) (4) (5) (6) (7) (8) (9),(10)
3,1.,1967,79131 47631 126762 22637 16192,38829
4,2.,1968,79159 47073 126232 24464 16224,40688


Pick 3rd Row for Column names

In [4]:
table2.columns = table2.iloc[2]

Drop the earlier columns

In [5]:
table2.drop([0,1,2],inplace=True)

Set column "(1)" as Dataframe Index

In [6]:
table2.set_index("(1)",inplace=True)

In [7]:
table2.tail(5)

2,(2),(3) (4) (5) (6) (7) (8) (9),(10)
(1),,,
45.,2011,302420 88464 390884 87839 47746,135585
46.,2012,306061 88921 394982 88453 46992,135445
47.,2013,312670 87847 400517 90543 44256,134799
48.,2014,354573 97078 106 451757 89129 42521 16,131666
49.,2015,328241 85169 47 413457 91528 42088 7,133623


Split the concatenated column "(3) (4) (5) (6) (7) (8) (9)" into its individual columns and conserve their column naming

In [8]:
table2[["(3)","(4)","(5)","(6)","(7)","(8)","(9)"]] = table2['(3) (4) (5) (6) (7) (8) (9)'].str.split(expand=True)\
                    .rename({0:"(3)",1:"(4)",2:"(5)",3:"(6)",4:"(7)",5:"(8)",6:"(9)"},axis=1)

Drop the concatenated column "(3) (4) (5) (6) (7) (8) (9)"

In [9]:
table2 = table2.drop('(3) (4) (5) (6) (7) (8) (9)',axis=1)

Re-order columns as per original PDF schema

In [10]:
col_order = ["(2)","(3)","(4)","(5)","(6)","(7)","(8)","(9)","(10)"]
table2 = table2[col_order]

In [11]:
table2.tail()

2,(2),(3),(4),(5),(6),(7),(8),(9),(10)
(1),,,,,,,,,
45.,2011,302420,88464,390884,87839,47746,None,None,135585
46.,2012,306061,88921,394982,88453,46992,None,None,135445
47.,2013,312670,87847,400517,90543,44256,None,None,134799
48.,2014,354573,97078,106,451757,89129,42521,16,131666
49.,2015,328241,85169,47,413457,91528,42088,7,133623


Notice that the values for the 6th, 7th and 8th Column are shifted to the left except for the last two rows.  
The 5th and 9th columns are missing values upto the year 2013.  
So basically, move the values of 5th, 6th and 7th to one column in the right.  
For all rows except the ones which are already filled with values.

In [12]:
table3 = table2.copy()
#Iterate throught the DataFrame row by row
for index, row in table3.iterrows():
    #Whenever the 9th Column has "None"
    if( str(row.loc["(9)"]) == "None" ):
        #Replace Missing values with NaN
        row.loc["(9)"] = np.nan
        #Move the rest of the columns to its right after typecasting as integer.
        row.loc["(8)"] = int(row.loc["(7)"])
        row.loc["(7)"] = int(row.loc["(6)"])
        row.loc["(6)"] = int(row.loc["(5)"])
        row.loc["(5)"] = np.nan

Rename the index according the PDF and typecast it to integer

In [13]:
table3.index = table3.index.set_names("Sl. No.").astype('float64').astype('int64')

Typecast each column to a numeric data type (either int64 or float64 )

In [28]:
for each in table3.columns:
    table3[each] = pd.to_numeric(table3[each])

In [29]:
table3.head()

Year Accidental Deaths                              Suicides  \
                           Male Female Transgender     Total     Male   
Sl. No.                                                                 
1        1967             79131  47631         NaN  126762.0  22637.0   
2        1968             79159  47073         NaN  126232.0  24464.0   
3        1969             82105  48650         NaN  130755.0  25947.0   
4        1970             89210  50542         NaN  139752.0  28846.0   
5        1971             68344  37257         NaN  105601.0  26326.0   

                                       
          Female Transgender    Total  
Sl. No.                                
1        16192.0         NaN  38829.0  
2        16224.0         NaN  40688.0  
3        17686.0         NaN  43633.0  
4        19582.0         NaN  48428.0  
5        17349.0         NaN  43675.0

Create Schema of Column-names from Original PDF in multi-level fashion

In [30]:
columns=[("Year",""),
         ('Accidental Deaths','Male'),
         ('Accidental Deaths','Female'),
         ('Accidental Deaths','Transgender'),
         ('Accidental Deaths','Total'),
         ('Suicides','Male'),
         ('Suicides','Female'),
         ('Suicides','Transgender'),
         ('Suicides','Total'),
        ]
columns

[('Year', ''),
 ('Accidental Deaths', 'Male'),
 ('Accidental Deaths', 'Female'),
 ('Accidental Deaths', 'Transgender'),
 ('Accidental Deaths', 'Total'),
 ('Suicides', 'Male'),
 ('Suicides', 'Female'),
 ('Suicides', 'Transgender'),
 ('Suicides', 'Total')]

Apply column-schema

In [31]:
table3.columns = pd.MultiIndex.from_tuples(columns)
table3

Year Accidental Deaths                              Suicides  \
                           Male Female Transgender     Total     Male   
Sl. No.                                                                 
1        1967             79131  47631         NaN  126762.0  22637.0   
2        1968             79159  47073         NaN  126232.0  24464.0   
3        1969             82105  48650         NaN  130755.0  25947.0   
4        1970             89210  50542         NaN  139752.0  28846.0   
5        1971             68344  37257         NaN  105601.0  26326.0   
6        1972             69838  36346         NaN  106184.0  26923.0   
7        1973             69613  36741         NaN  130654.0  25231.0   
8        1974             73580  37044         NaN  110624.0  27791.0   
9        1975             74987  38029         NaN  113016.0  26074.0   
10       1976             73098  38513         NaN  111611.0  24042.0   
11       1977             76013  41325         NaN  117338.0  23453.0   
12       1978             78352  40242         NaN  118594.0  24137.0   
13       1979             72448  36539         NaN  108987.0  22980.0   
14       1980             76293  40619         NaN  116912.0  24188.0   
15       1981             82328  39893         NaN  122221.0  23864.0   
16       1982             82977  43016         NaN  125993.0  26520.0   
17       1983             85577  42999         NaN  128576.0  27260.0   
18       1984             90636  43992         NaN  134628.0  29296.0   
19       1985             93568  46089         NaN  139657.0  30460.0   
20       1986             98480  48543         NaN  147023.0  31271.0   
21       1987            103727  48587         NaN  152314.0  34292.0   
22       1988            110235  53287         NaN  163522.0  37755.0   
23       1989            114953  54113         NaN  169066.0  40212.0   
24       1990            117932  56469         NaN  174401.0  43451.0   
25       1991            129142  58861         NaN  188003.0  46324.0   
26       1992            133977  60933         NaN  194910.0  47481.0   
27       1993            130499  61858         NaN  192357.0  49851.0   
28       1994            132241  58194         NaN  190435.0  52752.0   
29       1995            157219  65268         NaN  222487.0  52357.0   
30       1996            156106  63988         NaN  220094.0  51206.0   
31       1997            164876  69027         NaN  233903.0  56281.0   
32       1998            185520  72889         NaN  258409.0  61686.0   
33       1999            193652  78266         NaN  271918.0  65488.0   
34       2000            186324  69559         NaN  255883.0  66032.0   
35       2001            196129  74890         NaN  271019.0  66314.0   
36       2002            194850  65272         NaN  260122.0  69332.0   
37       2003            197285  62340         NaN  259625.0  70221.0   
38       2004            210190  67073         NaN  277263.0  72651.0   
39       2005            224806  69369         NaN  294175.0  72916.0   
40       2006            241210  73494         NaN  314704.0  75702.0   
41       2007            262918  77876         NaN  340794.0  79295.0   
42       2008            263809  78500         NaN  342309.0  80544.0   
43       2009            276333  80688         NaN  357021.0  81471.0   
44       2010            298262  86387         NaN  384649.0  87180.0   
45       2011            302420  88464         NaN  390884.0  87839.0   
46       2012            306061  88921         NaN  394982.0  88453.0   
47       2013            312670  87847         NaN  400517.0  90543.0   
48       2014            354573  97078       106.0  451757.0  89129.0   
49       2015            328241  85169        47.0  413457.0  91528.0   

                                        
          Female Transgender     Total  
Sl. No.                                 
1        16192.0         NaN   38829.0  
2        16224.0         NaN   40688.0  
3      

In [43]:
table3.to_csv("Outputs/CSV/Number_of_Accidental_Deaths_&_Suicides_in_India_During_1967_to_2015.csv")
table3.to_excel("Outputs/Excel/Number_of_Accidental_Deaths_&_Suicides_in_India_During_1967_to_2015.xlsx")